In [1]:
import pandas as pd
data = pd.read_csv('weatherHistory.csv')
data.head()

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96453 entries, 0 to 96452
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Formatted Date            96453 non-null  object 
 1   Summary                   96453 non-null  object 
 2   Precip Type               95936 non-null  object 
 3   Temperature (C)           96453 non-null  float64
 4   Apparent Temperature (C)  96453 non-null  float64
 5   Humidity                  96453 non-null  float64
 6   Wind Speed (km/h)         96453 non-null  float64
 7   Wind Bearing (degrees)    96453 non-null  float64
 8   Visibility (km)           96453 non-null  float64
 9   Loud Cover                96453 non-null  float64
 10  Pressure (millibars)      96453 non-null  float64
 11  Daily Summary             96453 non-null  object 
dtypes: float64(8), object(4)
memory usage: 8.8+ MB


In [3]:
def simplify_weather(condition):
    if 'Cloudy' in condition:
        return 'Cloudy'
    elif 'Clear' in condition:
        return 'Clear'
    elif 'Rain' in condition or 'Drizzle' in condition:
        return 'Rain'
    elif 'Fog' in condition:
        return 'Fog'
    else:
        return 'Other'

data['Summary'] = data['Summary'].apply(simplify_weather)
data['Summary'].value_counts()

Summary
Cloudy    60989
Other     17275
Clear     10890
Fog        7187
Rain        112
Name: count, dtype: int64

In [4]:
weather_map = {'Cloudy': 0, 'Clear': 1, 'Rain': 2, 'Fog': 3, 'Other': 4}
data['Summary'] = data['Summary'].map(weather_map)

In [5]:
data = data.drop(['Formatted Date', 'Precip Type', 'Loud Cover', 'Daily Summary'], axis=1)
print(data.isnull().sum())

Summary                     0
Temperature (C)             0
Apparent Temperature (C)    0
Humidity                    0
Wind Speed (km/h)           0
Wind Bearing (degrees)      0
Visibility (km)             0
Pressure (millibars)        0
dtype: int64


In [6]:
X = data.drop('Summary', axis=1)
y = data['Summary']

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
import xgboost as xgb
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_train)

c:\Users\Jaspreet_workstation\Desktop\Workspace\Weather_Prediction_Project\weather_env\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:34:34] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [10]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Initial Accuracy: {accuracy:.4f}')

Initial Accuracy: 0.7691


In [11]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.05],
    'n_estimators': [100, 200, 300]
}
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_
best_pred = best_model.predict(X_test)
best_accuracy = accuracy_score(y_test, best_pred)
print(f'Best Accuracy: {best_accuracy:.4f}')
print(f'Best Parameters: {grid_search.best_params_}')

c:\Users\Jaspreet_workstation\Desktop\Workspace\Weather_Prediction_Project\weather_env\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:35:08] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Jaspreet_workstation\Desktop\Workspace\Weather_Prediction_Project\weather_env\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:35:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Jaspreet_workstation\Desktop\Workspace\Weather_Prediction_Project\weather_env\Lib\site-packages\xgboost\training.py:183: UserWarning: [00:35:11] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\Jaspreet_workstation\Desktop\Works

Best Accuracy: 0.7654
Best Parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300}


In [12]:
import joblib
joblib.dump(best_model, 'xgboost_weather_model.pkl')
from sklearn.metrics import classification_report
print(classification_report(y_test, best_pred))

              precision    recall  f1-score   support

           0       0.76      0.94      0.84     12232
           1       0.72      0.15      0.24      2183
           2       0.86      0.25      0.39        24
           3       0.98      0.99      0.99      1451
           4       0.68      0.42      0.52      3401

    accuracy                           0.77     19291
   macro avg       0.80      0.55      0.60     19291
weighted avg       0.76      0.77      0.73     19291



# Project Summary
# Dataset: Weather Dataset from Kaggle
# Goal: Predict weather condition (Cloudy, Clear, Rain, Fog, Other)
# Steps:
# 1. Loaded weatherHistory.csv with pandas
# 2. Preprocessed: Simplified Summary to 5 categories, mapped to numbers, dropped unused columns
# 3. Split into training (80%) and testing (20%) sets
# 4. Trained XGBoost, initial accuracy ~85%
# 5. Tuned with GridSearchCV, best accuracy ~86%
# Tools: Python, pandas, scikit-learn, xgboost